In [1]:
import cv2
import numpy as np

In [2]:
img = cv2.imread('lenna.jpg')
cv2.imshow('lenna', img)
key = cv2.waitKey()
if key == 27:
    cv2.destroyAllWindows()

In [3]:
def show(img, name):
    cv2.imshow("%s" %(name), img)
    key = cv2.waitKey()
    if key == 27:
        cv2.destroyAllWindows()

# Gaussian Kernel Effect

In [4]:
g_img = cv2.GaussianBlur(img, (7, 7), 5)
show(g_img, name='gaussian_lenna')

In [5]:
# 图像变更模糊，因为范围更大，平均效果更明显
g_img = cv2.GaussianBlur(img, (17, 17), 5)
show(g_img, name='gaussian_lenna2')

In [6]:
# 图像更清晰，因为方差更小，高斯图像更尖锐，中心点起的作用更大
g_img = cv2.GaussianBlur(img,(7,7),1)
show(g_img, name='gaussian_lenna3')

In [7]:
# 来看看高斯核
kernel = cv2.getGaussianKernel(7, 5)
print(kernel)

[[0.12895603]
 [0.14251846]
 [0.15133131]
 [0.1543884 ]
 [0.15133131]
 [0.14251846]
 [0.12895603]]


In [8]:
# 为啥一维，因为一维运算快
# 理论解释
# 用显式地代码看隐式地高斯和显示地分步高斯地效果
g1_img = cv2.GaussianBlur(img,(7,7),5)
g2_img = cv2.sepFilter2D(img, -1, kernel, kernel) # ori, depth, kernelX, kernelY
show(g1_img, name='g1_blur_lenna')
show(g2_img, name='g2_blur_lenna')

# 2nd derivative: laplacian （双边缘效果）

In [9]:
kernel_lap = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]], np.float32)
lap_img = cv2.filter2D(img, -1, kernel=kernel_lap)
show(lap_img, name='lap_lenna')

# 图像锐化 = edge+ori

In [10]:
# app: sharpen
# 图像+edge=更锐利地图像，因为突出边缘
kernel_sharp = np.array([[0, 1, 0],[1, -3, 1],[0, 1, 0]], np.float32)
sharp_img = cv2.filter2D(img, -1, kernel=kernel_sharp)
show(sharp_img, name='sharp_lenna')

> 这样不对，因为，周围有4个1，中间是-3，虽然有边缘效果，但是周围的1会使得原kernel有滤波效果，使图像模糊；
\begin{bmatrix}
0 & 1 & 0 \\ 
1 & -3 & 1 \\
0 & 1 & 0
\end{bmatrix}

解决：所以取`kernel_sharp` 的相反数，再加上原图像，这样突出了中心像素，效果类似于小方差的高斯，所以可以既有边缘效果，又保留图像清晰度

In [11]:
kernel_sharp2 = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], np.float32)
sharp_img2 = cv2.filter2D(img, -1, kernel=kernel_sharp2)
show(sharp_img2, name='sharp_img2')

\begin{bmatrix}
0 & -1 & 0 \\ 
-1 & 5 & -1 \\
0 & -1 & 0
\end{bmatrix}

In [12]:
# 更“凶猛”的边缘效果
# 不仅考虑x-y方向上的梯度，同时考虑了对角线方向上的梯度
kernel_sharp = np.array([[1, 1, 1], [1, -8, 1], [1, 1, 1]], np.float32) 
sharp_img3 = cv2.filter2D(img, -1, kernel=kernel_sharp2)
show(sharp_img3, name='sharp_img3')

\begin{bmatrix}
1 & 1 & 1 \\ 
1 & -8 & 1 \\
1 & 1 & 1
\end{bmatrix}

In [13]:
######## Edge #########
# x轴
edgex = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], np.float32)
sharp_img4 = cv2.filter2D(img, -1, kernel=edgex)

# y轴
edgey = np.array([[-1, 0, -1], [-2, 0, 2], [-1, 0, 1]], np.float32)
sharp_img5 = cv2.filter2D(img, -1, kernel=edgey)
show(sharp_img4, name='edgex_lenna')
show(sharp_img5, name='edgey_lenna')

# 角点
* 关于角点的具体描述可以有几种：
    * 一阶导数(即灰度的梯度)的局部最大所对应的像素点；
    * 两条及两条以上边缘的交点；
    * 图像中梯度值和梯度方向的变化速率都很高的点；
    * 角点处的一阶导数最大，二阶导数为零，指示物体边缘变化不连续的方向。
 

In [14]:
img = cv2.imread('tower.jpg')
img = cv2.resize(img, (640, 480))
img_gray = np.float32(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))
print(img_gray)

img_harris = cv2.cornerHarris(img_gray, 2, 3, 0.05)    # 2： blockSize: window size; 3: Sobel kernel size
show(img_harris, name='img_harris ')

# 没法看原因：1. float类型； 2. img_harris本质上是每个pixel对于Harris函数的响应值
# 没有看的价值
print(img_harris)

# 为了显示清楚
# img_harris = cv2.dilate(img_harris , None)

thres = 0.05 * np.max(img_harris)
img[img_harris > thres] = [0, 0, 255]
show(img, name='img_harris')

[[ 72.  72.  72. ...  84.  84.  85.]
 [ 72.  72.  72. ...  84.  84.  83.]
 [ 72.  72.  73. ...  84.  84.  83.]
 ...
 [ 78.  77.  76. ... 212. 209. 197.]
 [113. 113. 105. ... 183. 193. 195.]
 [138. 155. 166. ... 184. 189. 201.]]
[[-4.8828126e-05 -4.8828126e-05 -7.8125007e-04 ...  0.0000000e+00
  -7.8125001e-04  8.5937493e-03]
 [-4.8828126e-05 -4.8828126e-05 -7.8125007e-04 ...  0.0000000e+00
  -7.8125001e-04  8.5937493e-03]
 [ 2.8808583e-03  2.8808583e-03 -2.8124999e-02 ...  0.0000000e+00
   1.2500000e-02  4.9999997e-02]
 ...
 [-2.0191997e+05 -2.0191997e+05 -2.2554295e+05 ...  3.5981750e+04
   4.9593257e+03 -2.4209869e+02]
 [-4.1817806e+05 -4.1817806e+05 -6.3627319e+05 ... -6.2683984e+03
  -7.2688477e+02  1.8453979e+02]
 [-2.9089394e+05 -2.9089394e+05 -3.8480744e+05 ...  3.1061951e+04
   1.6351560e+04  3.3497617e+03]]


# SIFT

In [15]:
########### SIFT ###########
img = cv2.imread('lenna.jpg')
# create sift class
sift = cv2.xfeatures2d.SIFT_create()
# detect SIFT
kp = sift.detect(img,None)   # None for mask
# compute SIFT descriptor
kp, des = sift.compute(img,kp)
print(des.shape)
img_sift= cv2.drawKeypoints(img,kp,outImage=np.array([]), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
show(img_sift, name='lenna_sift.jpg')

(1373, 128)


SIFT算法被申请了专利，将opencv版本退到3.4.2即可解决，卸载之前的包，然后
```shell
pip install opencv-python == 3.4.2.16
pip install opencv-contrib-python == 3.4.2.16
```